In [25]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [26]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ['EIN', 'NAME', 'STATUS', 'SPECIAL_CONSIDERATIONS'])
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1
...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,0,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,0,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,0,5000,0
34297,T5,Independent,C3000,ProductDev,Association,0,5000,1


In [27]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [28]:
# Look at APPLICATION_TYPE value counts for binning
app_type_count = application_df['APPLICATION_TYPE'].value_counts()
app_type_count


APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [29]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_type_count[app_type_count < 600].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: count, dtype: int64

In [30]:
# Look at CLASSIFICATION value counts for binning
#  YOUR CODE GOES HERE
class_type_count = application_df['CLASSIFICATION'].value_counts()
class_type_count.head(30)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
Name: count, dtype: int64

In [31]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_count = application_df['CLASSIFICATION'].value_counts()

# Filter value counts to include only counts greater than 1
class_count_gt1 = class_count[class_count > 1]

# Display the filtered value counts
print(class_count_gt1)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64


In [32]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_count[class_count < 300].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: count, dtype: int64

In [33]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_count[class_count < 300].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: count, dtype: int64

In [34]:
# Convert categorical data to numeric with `pd.get_dummies`
#  YOUR CODE GOES HERE
numeric_app_df = pd.get_dummies(application_df)
numeric_app_df

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,True,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
1,108590,1,False,False,True,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,5000,0,False,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,False,False
3,6692,1,False,False,True,False,False,False,False,False,...,True,False,False,True,False,False,False,False,False,False
4,142590,1,False,False,True,False,False,False,False,False,...,True,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,5000,0,False,False,False,True,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
34295,5000,0,False,False,False,True,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
34296,5000,0,False,False,True,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
34297,5000,1,False,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,False,False


In [35]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE
y = numeric_app_df["IS_SUCCESSFUL"]
X = numeric_app_df.drop(["IS_SUCCESSFUL"],axis=1)

# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [36]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)
# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [37]:
## Compile, Train and Evaluate the Model

In [39]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

# Added Third Layer
hidden_nodes_layer3 = 30

# Added Fourth Layer
hidden_nodes_layer4 = 30

nn1 = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="relu", input_dim=number_input_features))
# Second hidden layer
#  YOUR CODE GOES HERE
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu", input_dim=number_input_features))


# Third hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu", input_dim=number_input_features))


# Fourth hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu", input_dim=number_input_features))

# Output layer
#  YOUR CODE GOES HERE
nn1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# Check the structure of the model
nn1.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_14 (Dense)                     │ (None, 80)                  │           3,280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 30)                  │           2,430 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 30)                  │             930 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 30)                  │             930 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 1)                   │              31 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,601 (29.69 KB)

 Trainable params: 7,601 (29.69 KB)

 Non-trainable params: 0 (0.00 B)

In [40]:
# Compile the model
#  YOUR CODE GOES HERE
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [41]:
# Train the model
#  YOUR CODE GOES HERE
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 907us/step - accuracy: 0.7415 - loss: 0.5322
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 912us/step - accuracy: 0.7462 - loss: 0.5289
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 908us/step - accuracy: 0.7444 - loss: 0.5298
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 898us/step - accuracy: 0.7452 - loss: 0.5281
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 893us/step - accuracy: 0.7448 - loss: 0.5259
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 909us/step - accuracy: 0.7421 - loss: 0.5303
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 896us/step - accuracy: 0.7461 - loss: 0.5273
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 908us/step - accuracy: 0.7438 - loss: 0.5265
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 916us/step - accuracy: 0.7406 - loss: 0.5325
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 896us/step - accuracy: 0.7405 - loss: 0.5332
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 902us/step - accuracy: 0.7447 - loss: 0.5283
Epoch 12/100
804/80

804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 897us/step - accuracy: 0.7431 - loss: 0.5301
Epoch 68/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 899us/step - accuracy: 0.7414 - loss: 0.5311
Epoch 69/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 921us/step - accuracy: 0.7437 - loss: 0.5259
Epoch 70/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 896us/step - accuracy: 0.7377 - loss: 0.5344
Epoch 71/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 918us/step - accuracy: 0.7415 - loss: 0.5312
Epoch 72/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 925us/step - accuracy: 0.7469 - loss: 0.5269
Epoch 73/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 906us/step - accuracy: 0.7352 - loss: 0.5341
Epoch 74/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 911us/step - accuracy: 0.7392 - loss: 0.5361
Epoch 75/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 908us/step - accuracy: 0.7406 - loss: 0.5309
Epoch 76/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 909us/step - accuracy: 0.7451 - loss: 0.5284
Epoch 77/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 924us/step - accuracy: 0.7428 - loss: 0.5287
Epoch 78/100
804/804 ━━

In [42]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 1ms/step - accuracy: 0.7263 - loss: 0.5959
Loss: 0.5958583950996399, Accuracy: 0.7262973785400391


In [43]:
# Train the model reduced epochs to 50
#  YOUR CODE GOES HERE
fit_model1 = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 917us/step - accuracy: 0.7449 - loss: 0.5291
Epoch 2/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 940us/step - accuracy: 0.7488 - loss: 0.5221
Epoch 3/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 974us/step - accuracy: 0.7503 - loss: 0.5206
Epoch 4/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 944us/step - accuracy: 0.7416 - loss: 0.5298
Epoch 5/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 948us/step - accuracy: 0.7446 - loss: 0.5288
Epoch 6/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 944us/step - accuracy: 0.7442 - loss: 0.5314
Epoch 7/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 971us/step - accuracy: 0.7396 - loss: 0.5292
Epoch 8/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 939us/step - accuracy: 0.7443 - loss: 0.5283
Epoch 9/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 937us/step - accuracy: 0.7445 - loss: 0.5266
Epoch 10/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 927us/step - accuracy: 0.7449 - loss: 0.5273
Epoch 11/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 906us/step - accuracy: 0.7467 - loss: 0.5259
Epoch 12/50
804/804 ━━━━━━━━━━

In [45]:
# Evaluate the model using the test data
model_loss1, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 776us/step - accuracy: 0.7259 - loss: 0.5913
Loss: 0.5958583950996399, Accuracy: 0.7259474992752075


In [46]:
# Export our model to HDF5 file
nn1.save("AlphabetSoupCharity_Optimization.h5")